In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import xgboost as xgb
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
def ignore_warn(*args, **kwargs):
    pass
# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/araimairu-techjam/demographics.csv')
train_y = pd.read_csv('/kaggle/input/araimairu-techjam/train.csv')
kplus = pd.read_csv('/kaggle/input/araimairu-techjam/kplus.csv')
cc = pd.read_csv('/kaggle/input/araimairu-techjam/cc.csv')
eva = pd.read_csv('/kaggle/input/araimairu-techjam/test.csv')

kplus.tail(20)

In [ ]:
train.shape

เตรียมดำเนินการกับ Data ทั้งหมด

In [ ]:
all_data = train
all_data.tail(20)

In [ ]:
all_data.isnull().sum()

-นับจำนวนค่าของ ocp_cd แต่ละค่า

In [ ]:
all_data['ocp_cd'].value_counts()

นับจำนวนบัตรของแต่ละ id แล้วเก็บไว้ merge ที่หลัง

In [ ]:
TrainGroupBy= pd.DataFrame(all_data[['id','age']].groupby('id').count().reset_index())
TrainGroupBy = TrainGroupBy.rename(columns={"age": "CardCount"})

In [ ]:
TrainGroupBy.head()

In [ ]:
all_data.tail(10)

cc_no เชื่อมกับ id 

In [ ]:
cc

In [ ]:
all_dataTestMergeAll = pd.merge(all_data,cc,on='cc_no')
all_dataTestMergeAll.tail(5)

In [ ]:
print(all_dataTestMergeAll[all_dataTestMergeAll['id']==1 ])

In [ ]:
all_dataTestMergeAll.dtypes

![](http://)รวมค่า ใช้เงินของบัตรทั้งหมดไว้ใน id เดียว ดึงค่าทางสถิติต่างๆ sum std,etc

In [ ]:
CCTesTStD = pd.DataFrame()
CCTesTStD['CmeanSTD'] = all_dataTestMergeAll[['cc_txn_amt','id']].groupby('id').std().reset_index()['cc_txn_amt']
CCTesTStD['id'] = all_dataTestMergeAll[['cc_txn_amt','id']].groupby('id').std().reset_index()['id']
CCTesTStD['sum'] = all_dataTestMergeAll[['cc_txn_amt','id']].groupby('id').sum().reset_index()['cc_txn_amt']
print(CCTesTStD.sort_values(['CmeanSTD'],na_position='first'))
CCTesTStD['CmeanSTD'] = CCTesTStD['CmeanSTD'].fillna(CCTesTStD['sum'])
print(CCTesTStD.head())
CCTesTStD.isnull().sum()

In [ ]:
CCTesTStD.head(5)

In [ ]:
all_dataTestMergeAll.head()

pos_dt เป็นข้อมูลวันที่ของานใช้จ่าย ส่วน cc_txn_amt เป็นจำนวนเงินที่ใช้ในการใช้จ่าย
   เพื่อนที่จะใช้เป็นในการหาเงินเดือนจึงได้ นำเดือนใน pos_dt ที่มากที่สุดเพื่อหาช่วงกี่เดือนจากนั้นนำ cc_txn_amt รวมเข้าด้วยกันและหารด้วยจำนวนเดือนเพื่อหาค่าเฉลี่ยว่าเดือนนึงใช้จ่ายเดือนละเท่าใด
   และสถิติต่างๆ

In [ ]:
all_dataTestMergeAll['month'] = all_dataTestMergeAll['pos_dt'].apply  (lambda x :int(x.split('-')[1]) )
all_dataTestMergeAll_max = all_dataTestMergeAll[['month','id']].groupby('id').max().reset_index()
all_dataTestMergeAll_min = all_dataTestMergeAll[['month','id']].groupby('id').min().reset_index()


all_dataTestMergeAll_delta_month = all_dataTestMergeAll_max - all_dataTestMergeAll_min
all_dataTestMergeAll_delta_month.replace(0,1,inplace = True)
all_dataTestMergeAll_delta_month = all_dataTestMergeAll_delta_month.assign(id =all_dataTestMergeAll_max['id'])
print(all_dataTestMergeAll_delta_month.shape)


all_dataTestMergeAll_sum_cc_txn = all_dataTestMergeAll[['cc_txn_amt','id']].groupby('id').sum().reset_index()
all_dataTestMergeAll_delta_month['Cmean'] = all_dataTestMergeAll_sum_cc_txn['cc_txn_amt'] / all_dataTestMergeAll_delta_month['month'] 
all_dataTestMergeAll_delta_month['Cmax'] = all_dataTestMergeAll[['cc_txn_amt','id']].groupby('id').max().reset_index()['cc_txn_amt']
all_dataTestMergeAll_delta_month['Cmin'] = all_dataTestMergeAll[['cc_txn_amt','id']].groupby('id').min().reset_index()['cc_txn_amt']
all_dataTestMergeAll_delta_month['CmeanGB'] = all_dataTestMergeAll[['cc_txn_amt','id']].groupby('id').mean().reset_index()['cc_txn_amt']
all_dataTestMergeAll_delta_month['CmeanMinusSTD'] = CCTesTStD['CmeanSTD']-all_dataTestMergeAll_delta_month['Cmean']

CC_DATA =  all_dataTestMergeAll_delta_month.drop('month',axis = 1)
CC_DATA.tail(10)

รวม kplus เช่นเดียวกับ cc_no แต่ kplus มี id อยู่แล้วไม่จำเป็นต้องเชื่อมกับ id เหมือน cc_no

In [ ]:
print(kplus.head())

In [ ]:
kplus_delta_month = pd.DataFrame()
KpluSTesTStD = pd.DataFrame()

In [ ]:
KpluSTesTStD['KmeanSTD'] = kplus[['kp_txn_amt','id']].groupby('id').std().reset_index()['kp_txn_amt']
KpluSTesTStD['id'] = kplus[['kp_txn_amt','id']].groupby('id').std().reset_index()['id']
KpluSTesTStD['sum'] = kplus[['kp_txn_amt','id']].groupby('id').sum().reset_index()['kp_txn_amt']
KpluSTesTStD['Kmax'] = kplus[['kp_txn_amt','id']].groupby('id').max().reset_index()['kp_txn_amt']
print(KpluSTesTStD.sort_values(['KmeanSTD'],na_position='first'))

ข้อมูล STD บางรายการจะหายไปเนื่องจากมีข้อมูลเดียวจึงนำ ตาราง sum มาใส่แทนค่าที่ว่าง

In [ ]:
KpluSTesTStD['KmeanSTD'] = KpluSTesTStD['KmeanSTD'].fillna(KpluSTesTStD['sum'])
print(KpluSTesTStD.head())
KpluSTesTStD.isnull().sum()

ทำการเฉลี่ยรายจ่ายของแต่ละเดือน

In [ ]:
kplus['Kmonth'] = kplus['sunday'].apply(lambda x :int(x.split('-')[1]) )
kplus_max = kplus[['Kmonth','id']].groupby('id').max().reset_index()
kplus_min = kplus[['Kmonth','id']].groupby('id').min().reset_index()
kplus_delta_month['kmonth'] = kplus_max['Kmonth'] - kplus_min['Kmonth']
kplus_delta_month.replace(0,1,inplace = True)
kplus_sum_cc_txn = kplus[['kp_txn_amt','id']].groupby('id').sum().reset_index()
kplus_sum_cc_txn['Kmean'] = kplus_sum_cc_txn['kp_txn_amt'] / kplus_delta_month['kmonth']
kplus_sum_cc_txn['KmeanSTD'] = KpluSTesTStD['KmeanSTD']
kplus_sum_cc_txn['Kmax'] = kplus[['kp_txn_amt','id']].groupby('id').max().reset_index()['kp_txn_amt']
kplus_sum_cc_txn['Kmin'] = kplus[['kp_txn_amt','id']].groupby('id').min().reset_index()['kp_txn_amt']
kplus_sum_cc_txn['KmeanGB'] = kplus[['kp_txn_amt','id']].groupby('id').mean().reset_index()['kp_txn_amt']


KPLUS_DATA = kplus_sum_cc_txn.drop('kp_txn_amt',axis=1)
print(KPLUS_DATA.tail(30))

In [ ]:
all_data.head(5)

รวม CC_DATA KPLUS_DATA ALL_DATA

In [ ]:
all_data = pd.merge(all_data,CC_DATA,on='id',how = 'left')
all_data = pd.merge(all_data,KPLUS_DATA,on='id',how = 'left')
all_data = pd.merge(all_data,TrainGroupBy,on='id',how = 'left')

all_data[all_data['id']>50000].shape

-drop row ใน id ที่ซ้ำกัน

In [ ]:
all_data.drop_duplicates(['id'], inplace=True)

In [ ]:
all_data.drop('cc_no', axis=1,inplace = True)
all_data[all_data['id']>50000].shape

In [ ]:
all_data.isnull().sum()

1. -Turning categorical variables into quantitative variables 
1.    -get_dummies
1.    -concat
1.    -drop

In [ ]:
all_data['ocp_cd'] = all_data['ocp_cd'].fillna(0)
all_data['ocp_cd'] = all_data['ocp_cd'].astype(int)

In [ ]:
traindummies = pd.get_dummies(all_data['ocp_cd'])
traindummies.head()

In [ ]:
all_data = pd.concat([all_data,traindummies], axis=1)
all_data.head()

In [ ]:
all_data.drop('gender', axis=1,inplace = True)
all_data.head()

EDA 

In [ ]:
train_y

plot ข้อมูล age กับ income

In [ ]:
all_train_x = pd.merge(all_data,train_y,on = 'id')
fig, ax = plt.subplots()
ax.scatter(x = all_train_x['age'], y = all_train_x['income'])
plt.ylabel('income', fontsize=13)
plt.xlabel('age')
plt.show()


plot ข้อมูล occupation cd กับ income

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = all_train_x['ocp_cd'], y = all_train_x['income'])
plt.ylabel('income', fontsize=13)
plt.xlabel('ocp_cd')
plt.show()

drop outlier

In [ ]:
all_train_x.shape

In [ ]:
all_train_x.drop(all_train_x[(all_train_x['age']==2)&(all_train_x['income']>100000)].index,inplace=True)
all_train_x.drop(all_train_x[(all_train_x['age']==3)&(all_train_x['income']>470000)].index,inplace=True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==0)&(all_train_x['income']>200000)].index,inplace=True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==1) & (all_train_x['income']>300000)].index,inplace = True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==5) & (all_train_x['income']>270000)].index,inplace = True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==6) & (all_train_x['income']>100000)].index,inplace = True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==7) & (all_train_x['income']>360000)].index,inplace = True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==8) & (all_train_x['income']>100000)].index,inplace = True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==11) & (all_train_x['income']>220000)].index,inplace = True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==12) & (all_train_x['income']>250000)].index,inplace = True)
all_train_x.drop(all_train_x[(all_train_x['ocp_cd']==13) & (all_train_x['income']>410000)].index,inplace = True)

In [ ]:
all_train_x.shape

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = all_train_x['ocp_cd'], y = all_train_x['income'])
plt.ylabel('income', fontsize=13)
plt.xlabel('ocp_cd')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = all_train_x['age'], y = all_train_x['income'])
plt.ylabel('income', fontsize=13)
plt.xlabel('age', fontsize=13)
plt.show()

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(all_train_x['income'])

In [ ]:
all_train_x = all_train_x.drop(all_train_x[(all_train_x['income']>100000)&((all_train_x['Cmean']+all_train_x['Kmean'])<all_train_x['income']/2)].index)

In [ ]:
fig1, ax1 = plt.subplots()
ax1.set_title('Basic Plot')
ax1.boxplot(all_train_x[(all_train_x['ocp_cd']==3)]['income'])

In [ ]:
all_train_x.describe()

รวมค่าเฉลี่ยข้อมูลการใช้ credit card และ kplus

In [ ]:
all_train_x.columns.map(str)
all_train_x['CKmean']=all_train_x['Cmean']+all_train_x['Kmean']

plot CKmean กับ income

In [ ]:
sns.regplot(x="CKmean",y="income",data = all_train_x)
plt.ylim(0,)

เตรียมแยกข้อมูล all_train_x all_train_y ข้อมูลใช้สำหรับ train model
 all_test_x ใช้สำหรับเดาคำตอบส่ง

In [ ]:
all_train_y = all_train_x['income']
all_train_x.drop('income',axis=1,inplace = True)
all_train_x.drop('CKmean',axis=1,inplace = True)
#all_data.drop('ocp_cd', axis=1,inplace = True)
#all_data.drop('id', axis=1,inplace = True)

In [ ]:
all_data[all_data['id']>50000].shape

In [ ]:
all_eva_x = pd.merge(eva,all_data,on = 'id')
all_eva_x

In [ ]:
all_train_x

drop ข้อมูลที่ไม่ใช้ในการคำนวนเช่น id,ocp_cd

In [ ]:
all_train_x.drop('ocp_cd', axis=1,inplace = True)
all_eva_x.drop('ocp_cd', axis=1,inplace = True)
all_train_x.drop('id',axis=1,inplace=True)
all_eva_x.drop('id',axis=1,inplace=True)
all_train_x

นับคะแนนการเดาของ model ด้วย smape

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_predict,GridSearchCV
model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror')

In [ ]:
all_train_x.shape

In [ ]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(n_quantiles=10000, output_distribution='uniform', random_state=42)

In [ ]:
y_pred = cross_val_predict(model_xgb, all_train_x, all_train_y, cv=3)
print(smape(y_pred,all_train_y))

XGB model+Grid search**

In [ ]:
params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}

In [ ]:
#grid = GridSearchCV(model_xgb, params)
#grid.fit(all_train_x, all_train_y)

In [ ]:
#print(grid.best_estimator_)
from sklearn.externals import joblib
#joblib.dump(grid.best_estimator_, 'filename.pkl')

In [ ]:
#Y = grid.best_estimator_.predict(all_test_x)
loaded_model = joblib.load('/kaggle/input/pickle/pkl.pkl')
Y = loaded_model.predict(all_eva_x)

In [ ]:
y_pred = cross_val_predict(loaded_model, all_train_x, all_train_y, cv=3)
print(smape(y_pred,all_train_y))

In [ ]:
A_train ,A_test,B_train,B_test = train_test_split(all_train_x,all_train_y, test_size=0.2, random_state=42)

In [ ]:
B_train = qt.fit_transform(B_train.values.reshape((len(B_train), 1)))[:,0]

In [ ]:
B_test = qt.fit_transform(B_test.values.reshape((len(B_test), 1)))[:,0]

In [ ]:
model_xgb.fit(A_train, B_train)

In [ ]:
pred = cross_val_predict(model_xgb, A_test, B_test, cv=3)
print(smape(y_pred,all_train_y))

In [ ]:
model_xgb.feature_importances_
plt.bar(range(len(model_xgb.feature_importances_)), model_xgb.feature_importances_)
ind = np.arange(len(A_test.columns))
plt.xticks(ind, (A_test.columns))
plt.xticks(rotation=90)
plt.show()

ส่งคำตอบ

In [ ]:
eva['income'] = Y

In [ ]:
eva.to_csv('O_235.csv',index=False)

ดูความสำคัญของ feature แต่ละ feature

In [ ]:
# feature importance
print(loaded_model.feature_importances_)
# plot
plt.bar(range(len(loaded_model.feature_importances_)), loaded_model.feature_importances_)
plt.show()